In [1]:
import numpy as np
import pandas as pd
import redis

In [2]:
#redis-12833.c264.ap-south-1-1.ec2.cloud.redislabs.com:12833
#uDcCaTrv5WujZHanODLfW2yLSJSvNsqu
hostname = 'redis-18220.c256.us-east-1-2.ec2.cloud.redislabs.com'
portnumber = 18220
password = 'iqRWpxHl9SC9FuZ7G7Bv82kZ0NBQfSND'

r = redis.StrictRedis(host=hostname,
                      port=portnumber,
                      password=password)

In [3]:
r.ping()

True

In [4]:
#load the data (numpy zip)
np_file = np.load('dataframe_students_teacher.npz',allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values,columns=col_names)
df

,Name,Role,Facial_Features
0,ANGELINA JOLIE,STUDENT,"[0.43242034, 1.0748498, -0.12862365, -1.833291..."
1,BARACK OBAMA,TEACHER,"[0.94858104, -0.5584052, 0.094820715, -1.27965..."
2,CHRIS EVANS,STUDENT,"[0.7201715, -1.4684149, -0.7781253, -1.4601073..."
3,MORGAN FREEMAN,TEACHER,"[0.48535103, -0.6192555, -0.4069932, 1.0687788..."
4,SCARLETT JOHANSSON,STUDENT,"[0.19131963, -0.48843804, -1.690233, 0.4423967..."


In [5]:
df['name_role'] = df['Name']+'@'+df['Role']
df

,Name,Role,Facial_Features,name_role
0,ANGELINA JOLIE,STUDENT,"[0.43242034, 1.0748498, -0.12862365, -1.833291...",ANGELINA JOLIE@STUDENT
1,BARACK OBAMA,TEACHER,"[0.94858104, -0.5584052, 0.094820715, -1.27965...",BARACK OBAMA@TEACHER
2,CHRIS EVANS,STUDENT,"[0.7201715, -1.4684149, -0.7781253, -1.4601073...",CHRIS EVANS@STUDENT
3,MORGAN FREEMAN,TEACHER,"[0.48535103, -0.6192555, -0.4069932, 1.0687788...",MORGAN FREEMAN@TEACHER
4,SCARLETT JOHANSSON,STUDENT,"[0.19131963, -0.48843804, -1.690233, 0.4423967...",SCARLETT JOHANSSON@STUDENT


In [6]:
records = df[['name_role','Facial_Features']].to_dict(orient='records')
len(records)

5

In [7]:
#save dat in redis
#redis hashes
#keyname ='academy:register'

In [8]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial_Features']
    vector_bytes = vector.tobytes()
    r.hset(name='academy:register',key=name_role,value=vector_bytes)

In [9]:
##Registration Form

In [10]:
import cv2
from insightface.app import FaceAnalysis

In [11]:
faceapp = FaceAnalysis(name='buffalo_sc',
                       root='insightface_model',
                       providers=['CPUExecutionProvider'])
faceapp.prepare(ctx_id=0,det_size=(640,640),det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [12]:
#python input

person_name = input('Enter your name: ')

trials = 3
for i in range(trials):
    role = input("""
    Please choose
    1. Student
    2. Teacher
    Enter number either 1 or 2: """)
    
    if role in ('1', '2'):
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        break
    else:
        print('Invalid entry. Try again.')
        if i == trials - 1:
            print('Exceeds maximum trials')

key = person_name + '@'+ role
print('Your name = ',person_name)
print('Your role =',role)
print('Key =',key)


Your name =  Rabindra
Your role = Student
Key = Rabindra@Student


## collect Samples
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break
    results = faceapp.get(frame,max_num=1)
    for res in results:
        samples += 1
        x1,y1,x2,y2 = res['bbox'].astype(int)
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),1)

    if sample >= 200:
        break
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destoryAllWindows()

In [13]:
cap = cv2.VideoCapture(0)
face_embeddings = []
sample = 0

while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break

    results = faceapp.get(frame, max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
        embedding = res['embedding']
        face_embeddings.append(embedding)

    if sample >= 200:
        break

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()




In [14]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [15]:
x_mean_bytes = x_mean.tobytes()

In [16]:
r.hset(name='academy:register',key=key,value=x_mean_bytes)

1